In [3]:
##########################
#  Code Author: 1359531  #
##########################

In [4]:
from nlpfunctions import bow
from nlpfunctions import clean_up_sentence

In [5]:
import numpy as np
import random
from keras.models import load_model
import pickle
import json

Using TensorFlow backend.


In [6]:
data = pickle.load( open( "meta.pkl", "rb" ) )
words = data['words']
classes = data['classes']
with open('intents.json') as json_data:
    intents = json.load(json_data)
model = load_model('model.h5')

In [11]:
context = {}
ERROR_THRESHOLD = 0.3

def classify(sentence):
    p = np.array([bow(sentence, words)])
    results = list(model.predict(p))[0]
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def response(sentence, userID='anonymous',show_details=False):
    results = classify(sentence)
    output = {"response":"Sorry, please try again"}
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    if 'context_set' in i:
                        context[userID] = i['context_set']
                        output["context_set"] = i["context_set"]      
                    output["tag"] = i['tag']
                    if not 'context_filter' in i:
                        output["response"] = random.choice(i['responses'])
                    elif (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        #### Action Set ####
                        if 'action' in i:
                            print("Action:"+i['action'])
                        context.pop(userID, None)
                        output["response"] = random.choice(i['responses'])
                        output["context_filter"] = i["context_filter"]
            results.pop(0)
    
    if show_details: 
        return output
    else:
        return output["response"]

In [8]:
# while True:
#     try:
#         i = input()
#         if i == "exit":
#             break
#         else:
#             bot_response = response(i)
#             print("HeimDall: "+bot_response)
#     except (KeyboardInterrupt, EOFError, SystemExit):
#         break

In [12]:
while True:
    try:
        i = input()
        if i == "exit":
            break
        else:
            bot_response = response(i,show_details=True)
            print(bot_response)
    except (KeyboardInterrupt, EOFError, SystemExit):
        break

hello
{'response': 'Hello, Good to see you again', 'context_set': '', 'tag': 'greeting'}
gen cert
{'response': 'Do you want to generate certificate for Resource group or VM?', 'context_set': 'gen_cert', 'tag': 'gen_cert'}
vm
Action:gen_cert_type_vm
{'response': 'Okay, generating certificates for Virtual machine', 'tag': 'gen_cert_type_vm', 'context_filter': 'gen_cert'}
exit


In [18]:
type(response("hello",show_details=True))

import json
json.dumps(response("hello",show_details=True))

'{"response": "Hi there, how can I help?", "context_set": "", "tag": "greeting"}'